Selecting & indexing
====================

Basic `xarray` operations
-------------------------

In [ ]:
import xarray as xr

xr.set_options(display_expand_data=False)

First, let us generate some example data: a simple tight binding simulation of graphene.

In [ ]:
from erlab.io.exampledata import generate_data

dat = generate_data(seed=1).T

In [ ]:
dat

We have a three-dimensional array of intensity given in terms of $k_x$, $k_y$, and
binding energy. 


Let's extract a cut along $k_y = 0.3$.

In [ ]:
dat.sel(ky=0.3, method="nearest").plot()

Likewise, the Fermi surface can be extracted like this:

In [ ]:
dat.sel(eV=0.0, method="nearest").plot()

In [ ]:
cut = dat.sel(ky=0.3, method="nearest")
cut.sel(kx=slice(-0.2, 0.8), eV=slice(-0.25, 0.05)).plot()

In many scenarios, it is necessary to perform integration across multiple indices. This can be done by slicing and then averaging. The following code returns a new DataArray with the intensity integrated over a window of 50 meV centered at $E_F$.

In [ ]:
dat.sel(eV=slice(-0.025, 0.025)).mean("eV")

However, doing this every time is cumbersome, and we have lost the coordinate `eV`. In
the following sections, we introduce some utilities for convenient indexing.

The `qsel` accessor
-------------------

In [ ]:
dat.qsel(eV=0.0, eV_width=0.05)

In [ ]:
dat.qsel(kx=slice(-0.3, 0.3), ky=0.3, eV=0.0, eV_width=0.05)

In [ ]:
for radius in (0.03, 0.06, 0.09, 0.12):
    dat.qsel.around(radius, kx=0.52, ky=0.3).plot()

Masking
-------

Interpolation
-------------

In [ ]:
import erlab.plotting.erplot as eplt
import matplotlib.pyplot as plt
import numpy as np

a = 6.97
kx = [0, 2 * np.pi / (a * np.sqrt(3)), 2 * np.pi / (a * np.sqrt(3)), 0]
ky = [0, 0, 2 * np.pi / (a * 3), 0]


dat.qsel(eV=-0.2).qplot(aspect="equal", cmap="Greys")
plt.plot(kx, ky, "o-")

In [ ]:
import erlab.analysis as era

dat_sliced = era.interpolate.slice_along_path(
    dat, vertices={"kx": kx, "ky": ky}, step_size=0.01
)
dat_sliced

We can see that the data has been interpolated along the path. The new coordinate `path` contains the distance along the path, and the dimensions `kx` and `ky` are now expressed in terms of `path`.

The distance along the path can be calculated as the sum of the distances between consecutive points in the path.

In [ ]:
dat_sliced.qplot(cmap="Greys")
eplt.fermiline()

# Distance between each pair of consecutive points
distances = np.linalg.norm(np.diff(np.vstack([kx, ky]), axis=-1), axis=0)
seg_coords = np.concatenate(([0], np.cumsum(distances)))

plt.xticks(seg_coords, labels=["Γ", "M", "K", "Γ"])
plt.xlim(0, seg_coords[-1])
for seg in seg_coords[1:-1]:
    plt.axvline(seg, ls="--", c="k", lw=1)